<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TP-introduction-Pytorch" data-toc-modified-id="TP-introduction-Pytorch-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TP introduction Pytorch</a></span></li></ul></div>

# TP introduction Pytorch

Utiliser le tutoriel cifar10 sur un autre jeu de données

On a choisi ici [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist#loading-data-with-other-machine-learning-libraries), jeu de données composé d'un ensemble d'entraînement de 60k images niveaux de gris de 28x28 et 10k d'images de test

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

%matplotlib notebook

## Normalisation

In [ ]:
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True)

train_mean = trainset.data.float().mean()/255
train_std  = trainset.data.float().std()/255

In [ ]:
batch_size = 512 # taille de batch "optimisée" gpu

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize([train_mean], [train_std])])
    
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                       download=True, transform=transform)
    
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ['T-shirt','pantalon','pull','robe','manteau','sandale','shirt','sneaker','sac','boots']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img * train_std + train_mean     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images[:4]))
print(' '.join('%s' % classes[labels[j]] for j in range(4)))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# on a agrandi les couches du tutoriel et on a rajouté une couche de conv2D + maxpooling2d

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 128, 2)
        self.conv2 = nn.Conv2d(128, 64, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 32, 2)
        self.fc1 = nn.Linear(32 * 2 * 2, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))        
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 32 * 2 * 2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net().to(device)

In [ ]:
import torchsummary
torchsummary.summary(net,(1,28,28))

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from tqdm.notebook import tqdm
epochs = 20

bar_epoch = tqdm(total=epochs)
for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    bar = tqdm(total = len(trainloader))
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device),labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        bar.update()
        bar.set_description("batch loss %f" % loss.item())
        running_loss += loss.item() * len(data)
    bar_epoch.update()
    bar_epoch.set_description("epoch loss %f" % (running_loss/len(trainset)))

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
plt.figure()
imshow(torchvision.utils.make_grid(images[:4]))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j].item()] for j in range(4)))

In [ ]:
outputs = net(images.to(device))

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j].item()]
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))